Minhas anotações sobre o livro 'From Python to Numpy': https://www.labri.fr/perso/nrougier/from-python-to-numpy/

In [2]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt

In [3]:
#para mostrar todos os resultados e não apenas o último
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [4]:
#abre o site
import webbrowser

webbrowser.open('https://www.labri.fr/perso/nrougier/from-python-to-numpy/')

True

### Introdução

Um código otimizado em numpy muitas vezes será difícil de ler.

Recomenda-se comentar o código na hora que estiver programando. Depois de alguns dias já é capaz de esquecermos tudo

Se possível, colocar até figuras explicando o que se quer fazer

### 3 Anatomy of an Array

Revisão sobre as coisas básicas de layout de arrays. Fundamental para entender os demais capítulos do livro

O primeiro problema é limpar um array do tipo np.float32 da maneira mais rápida possível


In [5]:
Z = np.ones(4 * 1000000, np.float32)

#uma maneira é fazer assim
Z[...] = 0

In [6]:
Z

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [1]:
Z = np.ones(4 * 1000000, np.float32)
Z

NameError: name 'np' is not defined

#como os dados desse array são compatíveis com outros tipos de dados (ex: int8),
#podemos tentar limpar o array considerando que ele tem diferentes tipos de dados

#benchmark
Z = np.ones(4 * 1000000, np.float32)
%timeit Z.view(np.float32)[...] = 0

Z = np.ones(4 * 1000000, np.float32)
%timeit Z.view(np.float16)[...] = 0

Z = np.ones(4 * 1000000, np.float32)
%timeit Z.view(np.float64)[...] = 0

Z = np.ones(4 * 1000000, np.float32)
%timeit Z.view(np.int32)[...] = 0

Z = np.ones(4 * 1000000, np.float32)
%timeit Z.view(np.int16)[...] = 0

Z = np.ones(4 * 1000000, np.float32)
%timeit Z.view(np.int8)[...] = 0


#o autor diz que int8 foi o mais rápido para ele
#aqui o mais rápido foi float16

**Conclusão**: visualizar o array com o formato de dados (dtype) correto ajuda a acelerar a computação

O que é um array? Um array é um bloco de dados onde cada elemento pode ser acessado usando um determinado índice

Esse índice é determinado pelo formato do array (shape) e pelo tipo de data (dtype). Estes são os dois ingredientes necessários para fazer um array

In [9]:
z = np.arange(9).reshape(3,3).astype(np.int16)
z

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]], dtype=int16)

In [10]:
#tamanho dos elementos do array em bytes
z.itemsize

#shape
z.shape

#número de dimensões
z.ndim

2

(3, 3)

2

Depois dessa parte, o autor fala em stride e start offset e end offset.

Tem a ver com a quantidade de memória necessária para fazer algumas operações no vetor. Não entendi mais que isso.

Podemos visualizar os arrays usando diferentes layouts

In [11]:
#para ver a dimensão x
z.shape[0]

#para ver a dimensão de y
z.shape[1]

#para o tamanho de z
len(z)

3

3

3

In [12]:
#vamos repetir o processo acima para um array de shape diferente
y = np.arange(8).reshape(4,2).astype(np.int8)

y

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]], dtype=int8)

In [13]:
#dimensão x
y.shape[0]

#dimensão y
y.shape[1]

#tamanho de y
len(y)

#o len(vetor) sempre retorna o número de "linhas" de um vetor?

4

2

4

In [14]:
#se quisermos saber quantas células tem no array, usemos .size

y.size

8

In [15]:
#pausa para double colon
#vector[start:end:step]

x = np.arange(8)
x

#começando em 1 e terminando em 7
x[1:7]

#começando em 1, terminando em 7, pulando de 1 em 1 (deve dar o mesmo resultado acima)
x[1:7:1]

#começando em 1, terminando em 7, pulando de 2 em 2
x[1:7:2]

#considera a sequência toda, mas pega pulando de 2 em 2
x[::2]

#double colon sempre pega o primeiro elemento, cujo índice é zero

array([0, 1, 2, 3, 4, 5, 6, 7])

array([1, 2, 3, 4, 5, 6])

array([1, 2, 3, 4, 5, 6])

array([1, 3, 5])

array([0, 2, 4, 6])

In [16]:
#podemos também retirar um pedaço de Z
z

#pulando de dois em dois elementos, tanto nas linhas quanto nas colunas
z[::2,::2]


array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]], dtype=int16)

array([[0, 2],
       [6, 8]], dtype=int16)

In [17]:
#indexação usando ellipsis

#vamos buscar todos os primeiros elementos da quarta dimensão de um array
a = np.arange(16).reshape(2,2,2,2)

#usando ellipsis
a[...,0].flatten()

#usando notação padrão
a[:,:,:,0].flatten()

#resumindo: é uma maneira de economizar notação para arrays com várias dimensões

array([ 0,  2,  4,  6,  8, 10, 12, 14])

array([ 0,  2,  4,  6,  8, 10, 12, 14])

#### 3.3 Views and Copies

Qual a diferença entre indexing e fancy indexing?

1. indexing sempre retorn uma view do array
2. fancy indexing sempre retorna uma cópia (deep copy) do array

In [18]:
z = np.zeros(9)
z_view = z[:3]
z_view[:] = 1
z

#uma indexação alternativa é usar reticências (ellipsis) como índice. Isso retorna todos os elementos do array

array([1., 1., 1., 0., 0., 0., 0., 0., 0.])

In [19]:
#quando colocamos um par de chaves a mais, fazemos fancy indexing, ie, criamos uma cópia
# z_copy = z[[:3]] #repare que ele não aceita essa notação em fancy indexing

z = np.zeros(9)
z_copy = z[[0,1,2]]
z_copy[...] = 1
z

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

Como fancy indexing não aceita índices com ':', é uma boa prática criar variáveis para guardar os índices desejados

Ex.: index = [0,1,2] >> 
z[index]

In [20]:
z = np.zeros(9)

z_view = z[:3]
z_view


index = [0,1,2]
z_copy = z[index]
z_copy

array([0., 0., 0.])

array([0., 0., 0.])

E quando a gente não sabe se o array é uma cópia ou não? 

Checamos o atributo '.base'

In [21]:
z = np.zeros(9)
z_view = z[:3]
z_copy = z[[0,1,2]]


z_view.base is z

z_copy.base is z




True

False

In [22]:
#algumas funções do python retornam uma view quando possível (ex: ravel()), enquanto outras retornam copy (ex: flatten())

z = np.zeros((5,5))

z.ravel().base is z

z.flatten().base is z

#agora tentando com indexação
z[::2,::2].ravel().base is z
z[::2,::2].ravel().base is z[::2,::2]

#ou seja, ravel() retorna copy quando tem indexação (slicing) e view quando não tem



True

False

False

False

Cópias temporárias

Operações básicas com arrays geram cópias temporárias deles. Isso pode ser um problema que deixa o código mais lento

In [23]:
#exemplo 
x = np.ones(10, dtype = np.int)
y = np.ones(10, dtype = np.int)

a = 2 * x + 2 * y

#nesse caso, temos 3 arrays temporários: um para guardar 2*x, outro para 2*y e um terceiro para 2*x + 2*y


In [24]:
#como melhorar o código acima quando só precisamos do resultado final?

x = np.ones(10, dtype = np.int)
y = np.ones(10, dtype = np.int)

#multiplicando cada elemento de x e y por 2
x = np.multiply(x, 2, out = x)
y = np.multiply(y, 2, out = y)

#out é o resultado onde os cálculos são armazenados. Então nem precisava de x = ..., bastava out = x

#somando 
np.add(x, y, out = x)

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

#### 3.4 Conclusão

Exercício de conclusão: dados dois vetores z1 e z2, queremos saber se z1 é uma view de z2. Se sim, qual é essa view?

In [7]:
z1 = np.arange(10)
z1


#começa em 1, termina no núltimo (-1), e pula de dois em dois
z2 = z1[1:8:2] #equivalente a z1[1:-1:2]
z2

#note que se end = 8 ou end = 9, o resultado em z1 é igual. Acho que o autor do código não está preocupado com isso 

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

array([1, 3, 5, 7])

In [8]:
#a primeira parte é checar se z2 é uma view de z1
#fazemos isso usando o atributo .base
z2.base is z1

#True, então sabemos que z2 é view de z1

True

In [10]:
#primeira tentativa
z1 = np.arange(10)
z1

#começa em 1, termina no núltimo (-1), e pula de dois em dois
z2 = z1[1:8:2] #equivalente a z1[1:-1:2]
z2

def check_view(z1, z2):
    
    
    #check if z2 is a view of z1
    if(z2.base is z1):
        
        #extracts the index of the first element
        start = np.array(np.where(z1 == z2[0]))
        
        #extracts the index of the last element, them add it with the start to compare with the original array
        end = np.array(np.where(z1[::-1] == z2[-1]))
#         end = np.subtract(len(z1), end)
        
        
        #calculates the step based on the sizes of the arrays
        step = np.divide(len(z1[start.item():end.item()]), len(z2))
        step = np.array(np.ceil(step), dtype = np.int)
        
        
        #returns the desired arrays
        return start, end, step
        
    else:
        return print('z2 is not a view of z1')
        
        
    
#test
check_view(z1, z2)

#test with a deep copy, should return "not a view"
y2 = z1[[0,1,2]]
check_view(z1, y2)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

array([1, 3, 5, 7])

(array([[1]], dtype=int64), array([[2]], dtype=int64), array(1))

z2 is not a view of z1


In [11]:
#checando para outros dados
z3 = z1[1:8:3]


z1[:]
z1[::-1]
check_view(z1, z3)

#o resultado aqui está errado tanto no end quanto no step

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

(array([[1]], dtype=int64), array([[2]], dtype=int64), array(1))

In [33]:
#com step de tamanho 4
z1 = np.arange(10)
z1


z4 = z1[2:6:4]
z4

#acho que quando o resultado tem só um elemento, o step fica indefinido. 

#outras maneiras de achar z4:
#colocando mesmo stop = 6 e step > 4
z5 = z1[2:6:5]
z5

z6 = z1[2:6:6]
z6

#colocando stop < 6 e step < 4
y2 = z1[2:3:1]
y2





array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

array([2])

array([2])

array([2])

array([2])

Cheguei num ponto que não sei se está certo. Vou tentar replicar a função da solução para ver se ela dá conta de z4


In [12]:
def check_sol(z1,z2):
    #function based on the solution of the book
    if(z2.base is z1):
        
        step = z2.strides[0] // z1.strides[0]
        
        
        offset_start = np.byte_bounds(z2)[0] - np.byte_bounds(z1)[0]
        
        
        offset_stop = np.byte_bounds(z2)[-1] - np.byte_bounds(z1)[-1]
        
        
        start = offset_start // z1.itemsize
        
        stop = z1.size + offset_stop // z1.itemsize
        
        return start, stop, step
        
        
    else:
        return print('z2 is not a view of z1')
    
#test 1
z1 = np.arange(10)
z1

#começa em 1, termina no núltimo (-1), e pula de dois em dois
z2 = z1[1:8:2] #equivalente a z1[1:-1:2]
z2

check_sol(z1, z2)


#test 2
z4 = z1[2:6:4]
z4

check_sol(z1, z4)
print('onde é 3, deveria ser 6')

#dá um resultado estranho no stop. Vamos tentar melhorar isso
z4.itemsize
z4.strides


z1[2::4]
z1[2::4].itemsize
z1[2::4].strides


z1.strides
#os strides são iguais



    

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

array([1, 3, 5, 7])

(1, 8, 2)

array([2])

(2, 3, 4)

...

(16,)

array([2, 6])

4

(16,)

(4,)

In [13]:
z1.itemsize
z2.itemsize
z1.strides[0]
z2.strides

#strides diz pra gente quanto de memória a gente gasta para andar para a coluna ao lado e para a linha abaixo

4

4

4

(8,)

In [25]:

step = np.array(np.divide(z2.strides[0], z1.strides[0]),  dtype = np.int16)
step.item()

2

In [36]:
#vou incorporar strides na minha solução

#tudo junto
z1 = np.arange(10)
z1

#começa em 1, termina no núltimo (-1), e pula de dois em dois
z2 = z1[1:8:2] #equivalente a z1[1:-1:2]
z2



def check_view2(z1, z2):
#     breakpoint()
    
    #check if z2 is a view of z1
    if(z2.base is z1):
        
        #extracts the index of the first element
        start = np.array(np.where(z1 == z2[0]))
        
        #extracts the index of the last element, them add it with the start to compare with the original array
#         end = np.array(np.where(z1[::-1] == z2[-1]))
#         end = np.subtract(len(z1), end)
        
        #calculates the step based on the strides of the arrays
        #stride is the number of bytes required to transverse in each dimension of the array
        #ie, z1.strides = [4,20] means that it needs 4 bytes to move for the next column 
        #and 20 to move to the same position in the same row
        
        step = np.array(np.divide(z2.strides[0], z1.strides[0]),  dtype = np.int16)
#         step = np.divide(len(z1[start.item():end.item()]), len(z2))
#         step = np.array(np.ceil(step), dtype = np.int)
        
    
        #calculate end using the step
        end = np.multiply(len(z2),  step)
        end = np.add(end, start)
    
        
        #returns the desired arrays
        return start, end, step
        
    else:
        return print('z2 is not a view of z1')
        
        
    
#test
check_view2(z1, z2)

#replicating the view according to the function
z1[1:9:2]


#test with a deep copy
y2 = z1[[0,1,2]]
check_view2(z1, y2)

#test with z4
z4 = z1[2:6:4]
z4

check_view2(z1,z4)

#replicating the view according to the function
z1[2:6:4]


#test with negative step
print('test with negative step')

y1 = np.arange(9)
y1

y2 = y1[8:1:-1]
y2

check_view2(y1,y2)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

array([1, 3, 5, 7])

(array([[1]], dtype=int64), array([[9]], dtype=int64), array(2, dtype=int16))

array([1, 3, 5, 7])

z2 is not a view of z1


...

array([2])

test with negative step


array([0, 1, 2, 3, 4, 5, 6, 7, 8])

array([8, 7, 6, 5, 4, 3, 2])

(array([[8]], dtype=int64), array([[1]], dtype=int64), array(-1, dtype=int16))

Exercício adicional: expanda o código para os casos onde
1. Temos steps negativas
2. Comparamos arrays multi-dimensionais


A próxima tentativa é usar arrays multidimensionais, já que fizemos com steps negativas acima.


In [249]:
#depois do rascunho, estou pronto para fazer a função

def check_view3(z1, z2):
    
    #checa se z2 é view de z1 antes de prosseguir:
    if(z2.base is z1):
        
        
        #calculando step
        step = np.array(np.divide(z2.strides, z1.strides),  dtype = np.int16)
        
        #o jeito mais fácil de encontrar o start é comparar os primeiros elementos de a2 com as linhas equivalentes em a1
        #para que as dimensões sejam compatíveis, precisamos cortar z1 de acordo com as steps
        #criamos uma lista de índices para isso. ex: z1[::x,::y] equivale a z1[slice(None, None, x), slice(None, None,y)]
        
        ind = [slice(None, None, step[0])]

        for i in range(1,len(z2)):
            ind.append(slice(None, None, step[i]))
        
        #transforma em tuple para servir de argumento à z1
        ind = tuple(ind)
        
        start = np.array(np.where(z1[ind] == z2[...,0]), dtype = np.int16)
        
        #calculando end 
        
        end = np.multiply(np.array(z2.shape), step)
        end = np.add(end, start.T)
        
        #para evitar que o end seja negativo
        end = np.where(end < 0, None, end)
        
        return print(start, end, step)
        
        
    else:
        return print("z2 is not a view of z1")
    
    
#testando
a1 = np.arange(9).reshape(3,3).astype(np.int16)
a1

#significa que vai pegar todos os elementos do primeiro eixo, pulando de 2 em 2 (::2)
#e vai pegar todos os elementos do segundo eixo, pulando de 2 em 2 (::2)
a2 = a1[::2,::2]
a2

check_view3(a1, a2)

#testando se o resultado é igual à a2
a1[0:4:2, 0:4:2]

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]], dtype=int16)

array([[0, 2],
       [6, 8]], dtype=int16)

[[0]
 [0]] [[4 4]] [2 2]


array([[0, 2],
       [6, 8]], dtype=int16)

In [250]:
#testando check_view3 com arrays maiores e steps diferentes

z1 = np.arange(12).reshape(2,2,3).astype(np.int16)
z1


z2 = z1[:2:2, 1:3, ::3]
z2

check_view3(z1,z2)

#checando se dá no mesmo
z1[0:2:2,1:2:2, 0:3:3]

#dá sim

#testando com step negativo
z3 = z1[:2:2, 1:3, ::-1]
z3

check_view3(z1,z3)

#errou apenas porque disse que o end da terceira dimensão era negativo, consertei pedindo para retornar 'None'
z1[0:2:2, 1:2:1,2:None:-1]

array([[[ 0,  1,  2],
        [ 3,  4,  5]],

       [[ 6,  7,  8],
        [ 9, 10, 11]]], dtype=int16)

array([[[3]]], dtype=int16)

[[0]
 [1]
 [0]] [[2 2 3]] [2 1 3]


array([[[3]]], dtype=int16)

array([[[5, 4, 3]]], dtype=int16)

[[0]
 [1]
 [2]] [[2 2 None]] [ 2  1 -1]


array([[[5, 4, 3]]], dtype=int16)

### 4 Code Vectorization

### 5 Problem Vectorization

### 6 Custom Vectorization

### 7 Beyond Numpy

### 8 Conclusion

### Ideias para o código

1. Ao popular o último período do jogo com os valores de liquidação, posso usar ellipsis:
    * Ws_array[..., (T-1)] = Ls(T)
    
    
2. Checklist para acelarar código
    1. Diminuir ao máximo os arrays temporários